In [7]:
import parse_data.prepare_data as prepare_data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import globals
import data_strings
import data_extraction.extract_trial as extract_trial
import utils.cosine_similarity as cosine_similarity
import analysis.wall_visibility_and_choice as wall_visibility_and_choice
import data_extraction.get_indices as get_indices

### Notebook to identify opponent visibility at trial start 

In [ ]:
data_folder = data_strings.DATA_FOLDER
json_filenames_all_social = data_strings.JSON_FILENAMES_SOCIAL
json_filenames_all_solo = data_strings.JSON_FILENAMES_SOLO

In [ ]:
json_filenames = json_filenames_all_social

In [ ]:
df, trials_list = prepare_data.prepare_data(data_folder, json_filenames, combine=False)

Data is from period before 2024-09-13 00:00:00
Running dataframe through playerinfo_playerposition_conversion.
Loading complete.
Loading complete.
Preprocessing complete.
Preprocessing complete.


- Identify opponent position at trial start (so, this must be relative to a player)
- Identify angle to opponent at trial start (as with wall alcoves and sections in trajectory_headangle)
- Identify whether opponent is in field of view at trial start (as with wall visible)
- Filter down the list of trials to only include those in which a wall visility condition was True (either, one wall is visible at slice onset, or both are)
- Using the above as a condition to identify probability of choices (include in or take inspiration from wall_visibility_and_choice)
- extension: also include whether the opponent is facing towards or away from the player (i.e, identify angle of opponent to player at trial start)

Important points: make atomic functions for angle of one player to another, allow easy incorporation of whether opponent in facing player, avoid rewriting
code for conditions and probability. Wall separation should be included in these functions, but may not be used

##### First functions to write:
- Function that takes player id and specified trial and returns the position of the player at trial start (copy from trajectory extraction)
- Function that takes player and 'opponent' position at trial start, and the headangle of the player, and finds the angle from player to opponent (copy from trajectory headangle)
- Umbrella functions to repeat this process throughout a session for a specific player
- Function to filter down the array to only trials that fulfill a certain wall visibility condition (also record trial numbers for easier indexing of the list of 
  pandas dataframes)
- Identification of opponent being within FoV can be done with a boolean operation, no need for a function
- Apply function 2 and boolean operation to get whether player is in FoV of opponent (as a condition for the extension)
- Function to identify whether the choice condition was fulfilled on each of these 
- Function to calculate probability of making a certain choice (parameter) given the trials that fulfill the player and wall visibility condition, using the function to identify the trials that a particular choice was made

In [1]:
def get_player_position_trial_start(player_id, trial=None, trial_list=None, trial_index=None):
    ''' Return the position of the player at trial start.
        Takes the player_id and a single trial.'''
    
    trial = extract_trial.extract_trial(trial=trial, trial_list=trial_list, trial_index=trial_index)

    # get slice onset index, referenced to trial start
    slice_onset = trial[trial['eventDescription'] == globals.SLICE_ONSET]
    slice_onset_index = slice_onset.index - trial.index[0]
    
    # access the x and y locations stored in the player location dictionary indexed at the current player id
    x_coordinate = trial[globals.PLAYER_LOC_DICT[player_id]['xloc']].iloc[slice_onset_index]
    y_coordinate = trial[globals.PLAYER_LOC_DICT[player_id]['yloc']].iloc[slice_onset_index]

    player_position_coordinates = np.vstack([x_coordinate, y_coordinate])

    return player_position_coordinates


In [3]:
def get_player_headangle_vector_trial_start(player_id, trial=None, trial_list=None, trial_index=None):
    ''' Return the head angle of the player at trial start.
        Takes the player_id and a single trial. '''
    
    trial = extract_trial.extract_trial(trial, trial_list, trial_index)
    assert isinstance(trial, pd.DataFrame)

    # get slice onset index, referenced to trial start
    slice_onset = trial[trial['eventDescription'] == globals.SLICE_ONSET]
    slice_onset_index = slice_onset.index - trial.index[0]

    # find the euler angle for the rotation around the y (Unity vertical) axis
    y_rotation = trial[globals.PLAYER_ROT_DICT[player_id]['yrot']].iloc[slice_onset_index]

    # convert euler angle into euclidean vector
    x_component = np.sin(y_rotation)
    z_component = np.cos(y_rotation)
    head_angle_vector = np.vstack([x_component, z_component])

    return head_angle_vector



In [ ]:
def calculate_vector_norms_for_timepoint(self_other_vector, self_head_angle_vector):
    ''' Return the norm of the head angle vector and the self to other vector
        Returns scalar head_angle_vector_norm and self_to_other_vector_norm '''

    # find norm of headangle vector
    head_angle_vector_norm = np.linalg.norm(self_head_angle_vector)
    
    # find norm of the self-to-other vector
    self_to_other_vector_norm = np.linalg.norm(self_other_vector)

    return head_angle_vector_norm, self_to_other_vector_norm

In [ ]:
def calculate_angle_of_to_opponent_from_positions_and_headangle(self_position, other_position,
                                                                 self_head_angle_vector):
    ''' Return the cosine similarity (angle between vectors, length invariant) between the vector
        of self head angle and vector from self to other.
        Takes the self position, other position, and self head angle vector.'''

    # calculate euclidean vector from Self to Other
    # self and other position are size 2 vectors (x,y)
    self_other_vector = other_position - self_position

    # dot product between head angle vector and self-other vector
    dot_product_vectors = np.dot(self_other_vector, self_head_angle_vector)

    # vector norms for both self_other_vector and self_head_angle_vector 
    (self_other_vector_norm,
      self_head_angle_vector_norm) = calculate_vector_norms_for_timepoint(self_other_vector, 
                                                                          self_head_angle_vector)
    
    # cosine similarity between the two vectors
    vector_cosine_similarity = cosine_similarity.calculate_cosine_similarity_two_vectors(dot_product_vectors,
                                                                                         self_other_vector_norm,
                                                                                         self_head_angle_vector_norm)

    return vector_cosine_similarity
    



In [2]:
def get_two_player_positions_trial_start(player_id, trial=None, trial_list=None, trial_index=None):

    opponent_id = 1 if player_id == 0 else 1

    self_position = get_player_position_trial_start(player_id, trial=None, trial_list=None, trial_index=None)

    other_position = get_player_position_trial_start(opponent_id, trial=None, trial_list=None, trial_index=None)

    return self_position, other_position

In [ ]:
# Umbrella function to find trial start angle identification for a single player for one trial
def get_angle_of_opponent_from_player_trial(player_id, trial=None, trial_list=None, trial_index=None):
    ''' For a single trial, return the angle from player head direction to opponent player.
        Takes the player_id of Self, and the trial. '''

    # find self and other positions 
    self_position, other_position = get_two_player_positions_trial_start(player_id, trial=trial,
                                                                        trial_list=trial_list,
                                                                        trial_index=trial_index)

    # find general self head angle vector
    self_head_angle_vector = get_player_headangle_vector_trial_start(player_id, trial=trial,
                                                                    trial_list=trial_list,
                                                                    trial_index=trial_index)

    # calculate cosine similarity between self_head_angle_vector and self_to_other vector (which is calculated
    # in this function as the difference between Self and Other position)
    vector_cosine_similarity = calculate_angle_of_to_opponent_from_positions_and_headangle(self_position,
                                                                                          other_position,
                                                                                          self_head_angle_vector)
  
    return vector_cosine_similarity
    

In [ ]:
# Umbrella function to repeat trial start angle identification for a single player for one session

def get_angle_of_opponent_from_player_session(player_id, trial_list):
    ''' For all trials in a session, return the angle from player head direction to opponent player.
        Takes the player_id of Self (persistent throughout session) and the trial list. '''
    
    # get the angle for each trial in session, for a persistent Self player_id
    orientation_angle_to_other_session = np.full(len(trial_list), np.nan,  dtype=float)
    for i, trial in enumerate(trial_list):
    
        vector_cosine_similarity = get_angle_of_opponent_from_player_trial(player_id, trial=trial)

        orientation_angle_to_other_session[i] = vector_cosine_similarity

    return orientation_angle_to_other_session
    

In [ ]:
def filter_trials_one_wall_initially_visible(trial_list, player_id, wall_index):
    ''' Return a filtered trial list and list of indices from the original trial list that
        conform to a single trial wall being visible to player player_id at trial start,
        conferred by wall_index (e.g. 0 or 1 for wall1 or wall2) '''
    
    # find wall visibility for the full session
    (wall1_visible_session,
    wall2_visible_session) = wall_visibility_and_choice.get_walls_initial_visibility_session(trial_list=trial_list,
                                                                    player_id=player_id,
                                                                    debug=False, current_fov=110)
    
    if wall_index == 0:
        single_wall_visible_indices = np.where(wall1_visible_session == True)
    elif wall_index == 1:
        single_wall_visible_indices = np.where(wall2_visible_session == True)

    filtered_trial_list = trial_list[single_wall_visible_indices]

    return filtered_trial_list, single_wall_visible_indices


In [ ]:
def get_other_visible_session(orientation_angle_to_other_session, current_fov):
    ''' Return a boolean array for whether Other is visible to Self at trials start. '''

    # if Other is visible, the angle to orient Other into Self central view must be less than half the current
    # field-of-view.
    # At the threshold, Other enters visual periphery
    other_visible_session = orientation_angle_to_other_session < current_fov/2

    return other_visible_session


In [ ]:
def filter_trials_both_walls_initially_visible(trial_list, player_id):
    ''' Return a filtered trial list and list of indices from the original trial list that
        conform to both trials walls being visible to player player_id at trial start. '''
    
    # find wall visibility for the full session
    (wall1_visible_session,
    wall2_visible_session) = wall_visibility_and_choice.get_walls_initial_visibility_session(trial_list=trial_list,
                                                                    player_id=player_id,
                                                                    debug=False, current_fov=110)
    

    both_walls_visible_indices = np.where(wall1_visible_session == True and wall2_visible_session == True)

    filtered_trial_list = trial_list[both_walls_visible_indices]

    return filtered_trial_list, both_walls_visible_indices

In [6]:
# filter trial list further with visibility of Other

# then, work out which trials the player made a certain choice (choose High), use wall_visibility_and_choice

# calculate the basic probability

# (keep in mind being able to include Other visibility of Self)

In [ ]:
def filter_trials_other_visible(trial_list, other_visible_session):
    ''' Return a filtered trial list and list of indices from the original trial list that
        conform to Other visible to player player_id at trial start. '''
    
    other_visible_trial_indices = np.where(other_visible_session == True)

    filtered_trial_list = trial_list[other_visible_trial_indices]

    return filtered_trial_list, other_visible_trial_indices
    
    

In [ ]:
def placeholder(trial_list, player_id, given_wall_index=0):
    ''' Find trials where player choice (winner + loser, or just winner) aligned with
        the given wall index (e.g., 0 for wall1)'''
    
    # get player choice (wall number) for each trial
    # inferred choice can be used here
    player_choice = wall_visibility_and_choice.get_player_wall_choice(trial_list, player_id,
                                                                        inferred_choice=True, debug=False)

    given_wall_chosen_session = get_indices.was_given_wall_chosen(trial_list, player_choice,
                                                                  given_wall_index)
    
    # find the indices of the trials in trial_list where the given wall was chosen by player player_id
    given_wall_chosen_indices = np.where(given_wall_chosen_session == True)

    # filter and return trial_list
    return trial_list[given_wall_chosen_indices]


## DOUBLE CHECK THIS TOMORROW, WAS TIRED

    


### Sandbox-y

In [ ]:
def get_opponent_position_trial_start(player_id, trial=None, trial_list=None, trial_index=None):
    ''' Return the position of the opponent at trial start.
        Takes the player_id and a single trial.'''
    
    opponent_id = 0 if player_id == 1 else 0

    trial = extract_trial.extract_trial(trial=trial, trial_list=trial_list, trial_index=trial_index)

    # get slice onset index, referenced to trial start
    slice_onset = trial[trial['eventDescription'] == globals.SLICE_ONSET]
    slice_onset_index = slice_onset.index - trial.index[0]
    
    # access the x and y locations stored in the player location dictionary indexed at the current player id
    x_coordinate = trial[globals.PLAYER_LOC_DICT[opponent_id]['xloc']].iloc[slice_onset_index]
    y_coordinate = trial[globals.PLAYER_LOC_DICT[opponent_id]['yloc']].iloc[slice_onset_index]

    opponent_position_vector = np.vstack([x_coordinate, y_coordinate])

    return opponent_position_vector
